In [2]:
import pandas as pd

In [15]:
daily_corn = pd.read_csv("corn_prices_daily.csv", dtype={"date": "string"})
daily_corn["date"] = pd.to_datetime(daily_corn["date"].str.strip(), format="%d/%m/%Y")
daily_corn = daily_corn.sort_values("date").reset_index(drop=True)
daily_corn.head()

,date,Last Price
0,1992-01-02,2.41
1,1992-01-03,2.39
2,1992-01-06,2.40
3,1992-01-07,2.40
4,1992-01-08,2.43


In [3]:
weekly_probs_h1 = pd.read_csv("h1_probs.csv", parse_dates=["date"])
weekly_probs_h1.head()

,date,p_state0_h1,p_state1_h1,p_state2_h1,p_state3_h1
0,1992-01-05,0.035109,0.909816,0.054839,0.000236
1,1992-01-12,0.036271,0.911083,0.052502,0.000144
2,1992-01-19,0.033936,0.904253,0.061316,0.000494
3,1992-01-26,0.031881,0.865177,0.100857,0.002084
4,1992-02-02,0.027861,0.762242,0.203674,0.006222


In [19]:
# Copies (won't mutate originals)
wp = weekly_probs_h1.copy()   # has 'date' (Sundays) + probs
dc = daily_corn.copy()        # has 'date' (biz days) + price cols

# Ensure 'date' is a column and datetime in both
if 'date' not in wp.columns and isinstance(wp.index, pd.DatetimeIndex):
    wp = wp.reset_index().rename(columns={'index': 'date'})
if 'date' not in dc.columns and isinstance(dc.index, pd.DatetimeIndex):
    dc = dc.reset_index().rename(columns={'index': 'date'})

wp['date'] = pd.to_datetime(wp['date'])
dc['date'] = pd.to_datetime(dc['date'])

# Price columns to bring from daily_corn
price_cols = [c for c in dc.columns if c != 'date']

# Helper keys for the two attempts
wp['date_for_price2'] = wp['date'] - pd.Timedelta(days=2)
wp['date_for_price3'] = wp['date'] - pd.Timedelta(days=3)

# First merge: Sunday-2
dc2 = dc[['date'] + price_cols].rename(columns={'date': 'date_for_price2'})
out = pd.merge(wp, dc2, on='date_for_price2', how='left')

# Second merge: Sunday-3 (rename price cols to avoid clashes)
dc3 = dc[['date'] + price_cols].rename(columns={'date': 'date_for_price3'})
dc3 = dc3.rename(columns={c: f"{c}__alt" for c in price_cols})
out = pd.merge(out, dc3, on='date_for_price3', how='left')

# Fill NaNs from the alt columns (Thursday) where Friday was missing
for c in price_cols:
    out[c] = out[c].fillna(out[f"{c}__alt"])

# Clean up helper/alt columns; keep original Sunday date
out = (out
       .drop(columns=['date_for_price2', 'date_for_price3'] + [f"{c}__alt" for c in price_cols])
       .sort_values('date')
       .reset_index(drop=True))

print(out.head(300))



          date  p_state0_h1  p_state1_h1  p_state2_h1   p_state3_h1  \
0   1992-01-05     0.035109     0.909816     0.054839  2.360047e-04   
1   1992-01-12     0.036271     0.911083     0.052502  1.444166e-04   
2   1992-01-19     0.033936     0.904253     0.061316  4.944771e-04   
3   1992-01-26     0.031881     0.865177     0.100857  2.083921e-03   
4   1992-02-02     0.027861     0.762242     0.203674  6.222164e-03   
..         ...          ...          ...          ...           ...   
295 1997-08-31     0.117236     0.843769     0.038992  2.725129e-06   
296 1997-09-07     0.212439     0.753006     0.034553  2.167858e-06   
297 1997-09-14     0.414849     0.560031     0.025118  1.085512e-06   
298 1997-09-21     0.527012     0.453063     0.019924  1.312770e-06   
299 1997-09-28     0.627021     0.357708     0.015271  9.851720e-07   

     Last Price  
0          2.39  
1          2.43  
2          2.48  
3          2.55  
4          2.57  
..          ...  
295        2.63  
296

In [22]:
# Copies (so we don't mutate originals)
wp = weekly_probs_h1.copy()   # has 'date' (Sundays) and state probs (h=1)
dc = daily_corn.copy()        # has 'date' (business days) and price cols

# Ensure 'date' exists as column and is datetime
if 'date' not in wp.columns and isinstance(wp.index, pd.DatetimeIndex):
    wp = wp.reset_index().rename(columns={'index': 'date'})
if 'date' not in dc.columns and isinstance(dc.index, pd.DatetimeIndex):
    dc = dc.reset_index().rename(columns={'index': 'date'})

wp['date'] = pd.to_datetime(wp['date'])
dc['date'] = pd.to_datetime(dc['date'])

# Build join key: Sunday - 2 days (usually Friday)
wp['date_for_price'] = wp['date'] - pd.Timedelta(days=2)

# Prepare right table with the same key name
price_cols = [c for c in dc.columns if c != 'date']  # bring all non-date price cols
use_dc = dc[['date'] + price_cols].rename(columns={'date': 'date_for_price'})

# Left join on the helper key, keep Sunday 'date' from wp
data = pd.merge(wp, use_dc, on='date_for_price', how='left') \
         .drop(columns=['date_for_price']) \
         .sort_values('date').reset_index(drop=True)

print(data.head())


        date  p_state0_h1  p_state1_h1  p_state2_h1  p_state3_h1  Last Price
0 1992-01-05     0.035109     0.909816     0.054839     0.000236        2.39
1 1992-01-12     0.036271     0.911083     0.052502     0.000144        2.43
2 1992-01-19     0.033936     0.904253     0.061316     0.000494        2.48
3 1992-01-26     0.031881     0.865177     0.100857     0.002084        2.55
4 1992-02-02     0.027861     0.762242     0.203674     0.006222        2.57


In [24]:
# NaNs per column (count and % of rows)
nan_count_fri = data.isna().sum()
nan_pct_fri   = data.isna().mean().mul(100).round(2)

nan_summary = (
    pd.DataFrame({"n_nan": nan_count, "pct_nan": nan_pct})
      .sort_values("n_nan", ascending=False)
)

print(f"Shape: {data.shape[0]} rows × {data.shape[1]} cols")
print(nan_summary)

# If you also want the total NaNs across the whole DataFrame:
total_nans = int(data.isna().sum().sum())
print("Total NaNs in entire DataFrame:", total_nans)

Shape: 1752 rows × 6 cols
             n_nan  pct_nan
Last Price      84     4.79
date             0     0.00
p_state0_h1      0     0.00
p_state1_h1      0     0.00
p_state2_h1      0     0.00
p_state3_h1      0     0.00
Total NaNs in entire DataFrame: 84


In [25]:
# --- Inputs ---
wp = weekly_probs_h1.copy()   # weekly probs with Sunday 'date'
dc = daily_corn.copy()        # daily prices with business-day 'date'

# Ensure 'date' columns exist and are datetime
for df_name, df in [('wp', wp), ('dc', dc)]:
    if 'date' not in df.columns and isinstance(df.index, pd.DatetimeIndex):
        df.reset_index(inplace=True)
        df.rename(columns={'index':'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])

wp = wp.sort_values('date')
dc = dc.sort_values('date')

# Price columns to bring from daily_corn
price_cols = [c for c in dc.columns if c != 'date']
use_dc = dc[['date'] + price_cols].copy()

# ---------------------------
# Dataset A: Sunday-2 exact
# ---------------------------
wp_a = wp.copy()
wp_a['date_for_price2'] = wp_a['date'] - pd.Timedelta(days=2)
use_dc2 = use_dc.rename(columns={'date': 'date_for_price2'})
A = (pd.merge(wp_a, use_dc2, on='date_for_price2', how='left')
       .drop(columns=['date_for_price2'])
       .sort_values('date')
       .reset_index(drop=True))

A.to_csv('joined_sunday_minus2_exact.csv', index=False)

# --------------------------------------------
# Dataset B: Sunday-2 OR last available (≤ Sun)
# --------------------------------------------
# 1) As-of join gives last available price at/ before each Sunday
asof_tmp = pd.merge_asof(
    wp[['date']].sort_values('date'),
    use_dc.sort_values('date'),
    on='date', direction='backward'
)

# 2) Start from A, then fill NaNs with as-of values
asof_fill = asof_tmp.rename(columns={c: f"{c}__asof" for c in price_cols})
B = pd.merge(A, asof_fill[['date'] + [f"{c}__asof" for c in price_cols]], on='date', how='left')
for c in price_cols:
    B[c] = B[c].fillna(B[f"{c}__asof"])
B = B.drop(columns=[f"{c}__asof" for c in price_cols]).sort_values('date').reset_index(drop=True)

B.to_csv('joined_sunday_minus2_or_last.csv', index=False)

# ---------------------------
# NaN stats comparison
# ---------------------------
def nan_summary(df, cols=None):
    if cols is None:
        cols = df.columns
    nn = df[cols].isna().sum()
    pp = (df[cols].isna().mean() * 100).round(2)
    return pd.DataFrame({'n_nan': nn, 'pct_nan': pp}).sort_values('n_nan', ascending=False)

# 1) Compare ALL columns
all_A = nan_summary(A)
all_B = nan_summary(B)
compare_all = all_A.join(all_B, lsuffix='_A_exactFri', rsuffix='_B_fallback')
print("NaN summary (all columns):")
print(compare_all)

# 2) Compare PRICE columns only (what usually changes)
price_A = nan_summary(A, price_cols)
price_B = nan_summary(B, price_cols)
compare_price = price_A.join(price_B, lsuffix='_A_exactFri', rsuffix='_B_fallback')
print("\nNaN summary (price columns only):")
print(compare_price)

# Optional: grand totals
print("\nTotal NaNs — A (exact Fri):", int(A.isna().sum().sum()))
print("Total NaNs — B (fallback):", int(B.isna().sum().sum()))
print("\nSaved:\n - joined_sunday_minus2_exact.csv\n - joined_sunday_minus2_or_last.csv")


NaN summary (all columns):
             n_nan_A_exactFri  pct_nan_A_exactFri  n_nan_B_fallback  \
Last Price                 84                4.79                 0   
date                        0                0.00                 0   
p_state0_h1                 0                0.00                 0   
p_state1_h1                 0                0.00                 0   
p_state2_h1                 0                0.00                 0   
p_state3_h1                 0                0.00                 0   

             pct_nan_B_fallback  
Last Price                  0.0  
date                        0.0  
p_state0_h1                 0.0  
p_state1_h1                 0.0  
p_state2_h1                 0.0  
p_state3_h1                 0.0  

NaN summary (price columns only):
            n_nan_A_exactFri  pct_nan_A_exactFri  n_nan_B_fallback  \
Last Price                84                4.79                 0   

            pct_nan_B_fallback  
Last Price                 0

In [28]:
# --- Inputs (existing DataFrames) ---
wp = weekly_probs_h1.copy()   # weekly probs with Sunday 'date'
dc = daily_corn.copy()        # daily prices with business-day 'date'

# Ensure 'date' columns exist and are datetime
for df in (wp, dc):
    if 'date' not in df.columns and isinstance(df.index, pd.DatetimeIndex):
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])

wp = wp.sort_values('date')
dc = dc.sort_values('date')

# Price columns to bring from daily_corn
price_cols = [c for c in dc.columns if c != 'date']
use_dc = dc[['date'] + price_cols].copy()

# ---------------------------
# Dataset A: Sunday-2 exact
# ---------------------------
A = wp.copy()
A['date_for_price2'] = A['date'] - pd.Timedelta(days=2)
use_dc2 = use_dc.rename(columns={'date': 'date_for_price2'})
A = (pd.merge(A, use_dc2, on='date_for_price2', how='left')
       .sort_values('date')
       .reset_index(drop=True))

# Save A
A_out = A.drop(columns=['date_for_price2'])
A_out.to_csv('joined_sunday_minus2_exact.csv', index=False)

# -------------------------------------------------------
# Dataset B: Sunday-2 OR last available (≤ Sunday)
#   + add 'price_date_used' and 'fallback_days'
# -------------------------------------------------------
# 1) As-of join to get last available price and its date
use_dc_asof = use_dc.copy()
use_dc_asof['price_date'] = use_dc_asof['date']  # keep the matched price's own date

asof_tmp = pd.merge_asof(
    wp[['date']].sort_values('date'),
    use_dc_asof.sort_values('date'),
    on='date',
    direction='backward'
)

# 2) Start from A (Fri-only), then fill NaNs with as-of values
asof_fill = asof_tmp.rename(columns={c: f"{c}__asof" for c in price_cols})
asof_fill = asof_fill.rename(columns={'price_date': 'price_date__asof'})

B = pd.merge(A, asof_fill[['date', 'price_date__asof'] + [f"{c}__asof" for c in price_cols]],
             on='date', how='left')

# Fill price columns with as-of values where Friday was missing
for c in price_cols:
    B[c] = B[c].fillna(B[f"{c}__asof"])

# Determine which date was actually used for prices:
# If Friday (date_for_price2) had data, use that date; otherwise use as-of price_date.
has_fri = B[price_cols].notna().any(axis=1) & B['date_for_price2'].notna()
B['price_date_used'] = pd.NaT
B.loc[has_fri, 'price_date_used'] = B.loc[has_fri, 'date_for_price2']
B.loc[~has_fri, 'price_date_used'] = B.loc[~has_fri, 'price_date__asof']

# Fallback days = how far back from Sunday the used price is
B['fallback_days'] = (B['date'] - B['price_date_used']).dt.days

# Clean up helper columns (keep price_date_used & fallback_days)
drop_cols = [f"{c}__asof" for c in price_cols] + ['price_date__asof', 'date_for_price2']
B_out = (B.drop(columns=drop_cols)
           .sort_values('date')
           .reset_index(drop=True))

# Save B
B_out.to_csv('joined_sunday_minus2_or_last.csv', index=False)

# ---------------------------
# NaN stats comparison
# ---------------------------
def nan_summary(df, cols=None):
    if cols is None:
        cols = df.columns
    nn = df[cols].isna().sum()
    pp = (df[cols].isna().mean() * 100).round(2)
    return pd.DataFrame({'n_nan': nn, 'pct_nan': pp}).sort_values('n_nan', ascending=False)

print("NaN summary (ALL columns) — A (exact Fri):")
print(nan_summary(A_out), end="\n\n")

print("NaN summary (ALL columns) — B (fallback):")
print(nan_summary(B_out), end="\n\n")

print("NaN summary (PRICE columns only) — A vs B:")
price_A = nan_summary(A_out, price_cols)
price_B = nan_summary(B_out, price_cols)
print(price_A.join(price_B, lsuffix='_A_exactFri', rsuffix='_B_fallback'), end="\n\n")

# -------------------------------------------
# Collect & display big fallbacks (> 3 days)
# -------------------------------------------
dates_fallback_gt3 = B_out.loc[B_out['fallback_days'] > 2, 'date'].tolist()
print("Sundays with fallback_days > 2:")
print(dates_fallback_gt3)

# (Optional) also show their used price dates and offsets
if dates_fallback_gt3:
    detail_gt3 = B_out[B_out['fallback_days'] > 2][['date', 'price_date_used', 'fallback_days']]
    print("\nDetails for fallback_days > 2:")
    print(detail_gt3.head(30))


NaN summary (ALL columns) — A (exact Fri):
             n_nan  pct_nan
Last Price      84     4.79
date             0     0.00
p_state0_h1      0     0.00
p_state1_h1      0     0.00
p_state2_h1      0     0.00
p_state3_h1      0     0.00

NaN summary (ALL columns) — B (fallback):
                 n_nan  pct_nan
date                 0      0.0
p_state0_h1          0      0.0
p_state1_h1          0      0.0
p_state2_h1          0      0.0
p_state3_h1          0      0.0
Last Price           0      0.0
price_date_used      0      0.0
fallback_days        0      0.0

NaN summary (PRICE columns only) — A vs B:
            n_nan_A_exactFri  pct_nan_A_exactFri  n_nan_B_fallback  \
Last Price                84                4.79                 0   

            pct_nan_B_fallback  
Last Price                 0.0  

Sundays with fallback_days > 2:
[]
